In [1]:
from kubernetes import client, config

# Load the Kubernetes cluster configuration
config.load_kube_config()

# Create a CoreV1Api client instance
v1 = client.CoreV1Api()

def _get_pods_on_node(node_name, app_namespace='social-network2'):
    """
    Retrieve all pods scheduled on a given node and within the specified namespace.

    Parameters:
    - node_name (str): The name of the node to filter pods.
    - app_namespace (str): The namespace to search for pods.

    Returns:
    - list: A list of pod names in the given namespace and node.
    """
    # Initialize an empty list to store pod names
    pods_on_node = []

    # Get all pods in the specified namespace
    try:
        namespace_pods = v1.list_namespaced_pod(app_namespace, watch=False)

        # Loop through the pods and check if they are scheduled on the specified node
        for pod in namespace_pods.items:
            if pod.spec.node_name == node_name:
                # only returns string name
                # pods_on_node.append(pod.metadata.name)
                
                #retun the pod object
                pods_on_node.append(pod)

    except client.exceptions.ApiException as e:
        print(f"Error accessing namespace '{app_namespace}': {e}")

    return pods_on_node # retun pod object

        # Example Usage
        # node_name = 'k8s-worker-5'
        # app_namespace = 'social-network2'
        # pods = _get_pods_on_node(node_name, app_namespace)

        # print(f"Pods on node '{node_name}' in namespace '{app_namespace}': {pods}")
def _get_deployment_names_from_pods(pods, namespace):
    """
    Retrieve the deployment names associated with each pod in the provided list.

    Parameters:
    - pods (list): A list of pod objects.
    - namespace (str): The namespace in which the pods are located.

    Returns:
    - dict: A dictionary where keys are pod names, and values are their associated deployment names.
    """
    # Initialize a dictionary to store pod-to-deployment mappings
    pod_to_deployment = {}
    apps_v1 = client.AppsV1Api()


    for pod in pods:
        pod_name = pod.metadata.name
        owner_references = pod.metadata.owner_references

        # Check the owner references to see if this pod is managed by a ReplicaSet
        if owner_references:
            for owner in owner_references:
                if owner.kind == "ReplicaSet":
                    # Retrieve the ReplicaSet by name
                    replicaset_name = owner.name
                    replicaset = apps_v1.read_namespaced_replica_set(replicaset_name, namespace)

                    # Extract the Deployment name (it's typically the owner of the ReplicaSet)
                    rs_owner_references = replicaset.metadata.owner_references
                    if rs_owner_references:
                        for rs_owner in rs_owner_references:
                            if rs_owner.kind == "Deployment":
                                deployment_name = rs_owner.name
                                pod_to_deployment[pod_name] = deployment_name
                                break

    return pod_to_deployment

# Example usage:

node_name = 'k8s-worker-1'
# Example pod objects fetched using kubernetes client or passed externally
v1 = client.CoreV1Api()
pods = _get_pods_on_node(node_name, app_namespace = 'social-network2')


# Get the deployment names from the pod list
pod_deployment_map = _get_deployment_names_from_pods(pods, namespace = 'social-network2')

for pod, deployment in pod_deployment_map.items():
    print(f"Pod: {pod}, Deployment: {deployment}")


Pod: post-storage-service-84b659876c-mffc4, Deployment: post-storage-service
Pod: social-graph-mongodb-78b95799d7-zdh4c, Deployment: social-graph-mongodb
Pod: url-shorten-mongodb-5697666656-4nk42, Deployment: url-shorten-mongodb
Pod: user-mention-service-764c7d97c5-qp2kr, Deployment: user-mention-service
Pod: user-timeline-redis-6bf758cc4f-924jd, Deployment: user-timeline-redis


In [2]:
pod_deployment_map

{'post-storage-service-84b659876c-mffc4': 'post-storage-service',
 'social-graph-mongodb-78b95799d7-zdh4c': 'social-graph-mongodb',
 'url-shorten-mongodb-5697666656-4nk42': 'url-shorten-mongodb',
 'user-mention-service-764c7d97c5-qp2kr': 'user-mention-service',
 'user-timeline-redis-6bf758cc4f-924jd': 'user-timeline-redis'}

In [5]:
from kubernetes import client, config

# Load Kubernetes configuration (local kubeconfig or in-cluster config)
config.load_kube_config()

# Create a CoreV1Api client instance
v1 = client.CoreV1Api()

def get_worker_node_names():
    """
    Retrieve a list of all worker node names in the Kubernetes cluster, excluding master nodes.

    Returns:
    - list: A list containing the names of all worker nodes.
    """
    worker_nodes = []

    try:
        # Get all nodes in the cluster
        all_nodes = v1.list_node().items

        # Loop through each node and check if it's a worker node
        for node in all_nodes:
            node_name = node.metadata.name

            # Check node labels to exclude master/control-plane nodes
            labels = node.metadata.labels
            if 'node-role.kubernetes.io/master' not in labels and 'node-role.kubernetes.io/control-plane' not in labels:
                worker_nodes.append(node_name)

    except client.exceptions.ApiException as e:
        print(f"Error retrieving nodes: {e}")

    return worker_nodes

# Example usage:
worker_nodes = get_worker_node_names()
print(f"Worker Nodes: {worker_nodes}")


Worker Nodes: ['k8s-worker-1', 'k8s-worker-2', 'k8s-worker-3', 'k8s-worker-4', 'k8s-worker-5', 'k8s-worker-6', 'k8s-worker-7', 'k8s-worker-8', 'k8s-worker-9']


In [3]:
from kubernetes import client, config

# Load Kubernetes configuration (local or in-cluster)
config.load_kube_config()

# Create an AppsV1Api client instance
apps_v1 = client.AppsV1Api()

def _get_deployment_from_pod(pod, namespace):
    """
    Retrieve the deployment name associated with a given pod.

    Parameters:
    - pod (V1Pod): The Kubernetes pod object.
    - namespace (str): The namespace where the pod is located.

    Returns:
    - str: The name of the deployment managing this pod, or None if not applicable.
    """
    # Check the pod's owner references to see if it's managed by a ReplicaSet
    owner_references = pod.metadata.owner_references
    if owner_references:
        for owner in owner_references:
            if owner.kind == "ReplicaSet":
                # Retrieve the ReplicaSet by name
                replicaset_name = owner.name
                try:
                    replicaset = apps_v1.read_namespaced_replica_set(replicaset_name, namespace)

                    # Extract the deployment name from the ReplicaSet's owner references
                    rs_owner_references = replicaset.metadata.owner_references
                    if rs_owner_references:
                        for rs_owner in rs_owner_references:
                            if rs_owner.kind == "Deployment":
                                return rs_owner.name
                except client.exceptions.ApiException as e:
                    print(f"Error retrieving ReplicaSet '{replicaset_name}': {e}")
    return None

# Example usage:
# Assuming you have the pod object already fetched or passed as an argument
namespace = 'social-network'
v1 = client.CoreV1Api()
pod_name = 'compose-post-service-866f6d7b74-d6wnb'
pod = v1.read_namespaced_pod(name=pod_name, namespace=namespace)

# Get the deployment name associated with this pod
deployment_name = _get_deployment_from_pod(pod, namespace)
if deployment_name:
    print(f"The pod '{pod_name}' is managed by the deployment '{deployment_name}'.")
else:
    print(f"No deployment found for the pod '{pod_name}'.")


The pod 'compose-post-service-866f6d7b74-d6wnb' is managed by the deployment 'compose-post-service'.


In [ ]:
# from prometheus_api_client import PrometheusConnect
# import pandas as pd

# # Variables to configure
# PROMETHEUS_URL = prom_url
# TARGET_POD = "compose-post-service-866f6d7b74-45swb"
# TARGET_NAMESPACE = "social-network"
# QUERY_WINDOW = "1m"  # Specify your query window here

# # Connect to Prometheus
# prometheus = PrometheusConnect(url=PROMETHEUS_URL, disable_ssl=True)

# # Prometheus query to find all upstream pods sending traffic to the target pod
# query = f'''
# sum by (source_workload, source_workload_namespace) (rate(istio_requests_total{{destination_pod="{TARGET_POD}", destination_workload_namespace="{TARGET_NAMESPACE}"}}[{QUERY_WINDOW}]))
# '''

# # Execute the query
# data = prometheus.custom_query(query)

# # Parse the result
# if data:
#     traffic_data = []
#     for entry in data:
#         source_workload = entry["metric"]["source_workload"]
#         source_namespace = entry["metric"]["source_workload_namespace"]
#         traffic_volume = float(entry["value"][1])
#         traffic_data.append({
#             "source_workload": source_workload,
#             "source_namespace": source_namespace,
#             "traffic_volume": traffic_volume
#         })

#     # Convert to a DataFrame for analysis or recording
#     traffic_df = pd.DataFrame(traffic_data)
#     print(traffic_df)

#     # Optionally, save to a CSV or other storage format
#     traffic_df.to_csv("upstream_pod_traffic.csv", index=False)
# else:
#     print("No traffic data found.")